<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C4-Web_Hackery/Mapping_Web_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Web Hackery__*

The ability to analyze web applications is an absolutely critical skill for any attacker or penetration tester. In most modern networks, web applications present the largest attack surface and therefore are also the most common avenue for gaining access to the web applications themselves.

You'll find a number of excellent web application tools written in Python, including __w3af__ and __sqlmap__. Quite frankly, topics such as _SQL injection_ have been beaten to death, and the tooling available is mature enough that we don't need to reinvent the wheel. Instead, we'll explore the basics of interacting with the web by using Python and then build on this knowledge to create reconnaissance and brute-force tooling. By creating a few different tools, you should learn the fundamental skills you need to build any type of web application assessment tool that your particular attack scenario calls for.

In this chapter, we'll look at three scenarios for attacking a web app. In the first scenario, you know the web framework that the target uses, and that framework happens to be open source. A web app framework contains many files and directories within directories within directories. We'll create a map that shows the hierarchy of the web app locally and use that information to locate the real files and directories on the live target.

In the second scenario, you know only the URL for your target, so we'll resort to brute-forcing the same kind of mapping by using a word list to generate a list of filepaths and directory names that may be present on the target. We'll then attempt to connect to the resulting list of possible paths against a live target.

In the third scenario, you know the base URL of your target and its login page. We'll examine the login page and use a word list to brute-force a login.

### *__Using Web Libraries__*

We'll start by going over the libraries you can use to interact with web services. When performing network-based attacks, you may be using your own machine or a machine inside the network you're attacking. If you are on a compromised machine, you'll have to make do with what you've got, which might be a bare-bones Python 2.x or Python 3.x installation. We'll take a look at what you can do in those situations using the standard library. For the remainder of the chapter, however, we'll assume you're on your attacker machine using the most up-to-date packages.

### *__The urllib2 Library for Python 2.x__*

You'll see the __urllib2__ library used in code written for Python 2.x. It's bundled into the standard library. Much like the __socket__ library for writing network tooling, people use the __urllib2__ library when creating tools to interact with web services. Let's take a look at code that makes a very simple _GET_ request to the No Starch Press website:

In [ ]:
import urllib2

url = "https://www.nostarch.com"
# GET
response = urllib2.urlopen(url) #[1]
print(response.read()) #[2]
response.close()

This is the simplest example of how to make a _GET_ request to a website. We pass in a URL to the __urlopen__ function __[1]__, which returns a file-like object that allows us to read back the body of what the remote web server returns __[2]__. As we're just fetching the raw page from the No Starch website, no JavaScript or other client-side languages will execute.

In most cases, however, you'll want more fine-grained control over how you make these requests, including being able to define specific headers, handle cookies, and create _POST_ requests. The __urllib2__ library includes a __Request__ class that gives you this level of control. The following example shows you how to create the same _GET_ request by using the __Request__ class and by defining a custom __User-Agent__ HTTP header:

In [ ]:
import urllib2

url = "https://www.nostarch.com"
headers = {"User-Agent": "Googlebot"} #[1]

request = urllib2.Request(url, headers=headers) #[2]
response = urllib2.urlopen(request) #[3]

print(response.read())
response.close()

## *__Mapping Open Source Web App Installations__*



### *__Mapping the WordPress Framework__*

